In [110]:
from autumn.projects.sm_covid2.common_school.project_maker import get_school_project_parameter_set
import pandas as pd
from copy import copy
from pathlib import Path

In [111]:
params_info = pd.read_excel("params_info.xlsx")

In [112]:
iso3 = "FRA"
first_date_with_death = 0
sero_age_min = 0
sero_age_max = 100
analysis='main'

In [113]:
model_params = get_school_project_parameter_set(
    iso3,
    first_date_with_death,
    sero_age_min,
    sero_age_max,
    analysis
).baseline

In [114]:
def get_param_val(p_name, model_params):
    if "." not in p_name:
        return model_params[p_name]
    else:
        nested_par = model_params
        nested_keys = p_name.split(".")
        for k in nested_keys:
            nested_par = nested_par[k]        
        return nested_par

In [115]:
def make_output_params_df(params_info, model_params):
    params_df = copy(params_info)

    for index in params_df.index:
        if params_df.loc[index, 'value'] == "LOAD":
            params_df.loc[index, 'value'] = get_param_val(params_df.loc[index, 'param_key'], model_params)

    return params_df

In [116]:
df = make_output_params_df(params_info, model_params)

In [117]:
tex_table = df.to_latex(
    columns=['legible_name', 'value', 'unit', 'evidence'],
    header=['Parameter', "Value", "Unit", "Evidence"],
    index=False
)

In [118]:
tex_table = tex_table.replace("\\textbackslash cite\\", r"\cite").replace("\\}", "}")

In [119]:
out_path = Path.home() / "Models"/ "AuTuMN_new" / "docs" / "tex" / "tex_descriptions" / "projects" / "sm_covid" / "param_table.tex"

In [120]:
with out_path.open("w") as f:
    f.write(tex_table)